# 1. Library import

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data

import time
import numpy as np
import gc
import sys
import pickle
import copy
import pandas as pd
from AttentiveLayers_MLP import *
from Featurizer import *
from getFeatures import *

from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc

from rdkit import Chem
from rdkit.Chem import QED
from numpy.polynomial.polynomial import polyfit
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
from IPython.display import SVG, display
import seaborn as sns
sns.set(color_codes = True)

In [2]:
import os
os.getcwd()

'/home/dohyeon/hERGAT'

In [3]:
import os
os.chdir('/home/dohyeon/hERGAT/dataset')

# dataset 불러오기 

## Canonical smiles를 기준으로 중복 데이터 삭제

In [4]:
task_name = 'Class'
tasks = ['Class']
raw_filename = "hERG dataset.csv"
feature_filename = raw_filename.replace('.csv','.pickle')
filename = raw_filename.replace('.csv','')
prefix_filename = raw_filename.split('/')[-1].replace('.csv','')
smiles_tasks_df = pd.read_csv(raw_filename)
smilesList = smiles_tasks_df.SMILES.values
print("number of all smiles: ",len(smilesList))
atom_num_dist = []
remained_smiles = []
canonical_smiles_list = []
for smiles in smilesList:
    try:        
        mol = Chem.MolFromSmiles(smiles)
        atom_num_dist.append(len(mol.GetAtoms()))
        remained_smiles.append(smiles)
        canonical_smiles_list.append(Chem.MolToSmiles(Chem.MolFromSmiles(smiles), isomericSmiles=True))
    except:
        print("not successfully processed smiles: ", smiles)
        pass
print("number of successfully processed smiles: ", len(remained_smiles))
smiles_tasks_df = smiles_tasks_df[smiles_tasks_df["SMILES"].isin(remained_smiles)]
# print(smiles_tasks_df)
smiles_tasks_df['cano_smiles'] =canonical_smiles_list
assert canonical_smiles_list[8]==Chem.MolToSmiles(Chem.MolFromSmiles(smiles_tasks_df['cano_smiles'][8]), isomericSmiles=True)
smiles_tasks_df.head()

number of all smiles:  7933
number of successfully processed smiles:  7933


,Unnamed: 0,SMILES,pIC50,hERG,Class,cano_smiles
0,0,Cc1ccc(CN2[C@@H]3CC[C@H]2C[C@@H](C3)Oc4cccc(c4...,9.85,0.979,1,Cc1ccc(CN2[C@@H]3CC[C@H]2C[C@H](Oc2cccc(C(N)=O...
1,1,COc1nc2ccc(Br)cc2cc1[C@@H](c3ccccc3)[C@@](O)(C...,9.70,0.994,1,COc1nc2ccc(Br)cc2cc1[C@@H](c1ccccc1)[C@@](O)(C...
2,2,NC(=O)c1cccc(O[C@@H]2C[C@H]3CC[C@@H](C2)N3CCCc...,9.60,0.986,1,NC(=O)c1cccc(O[C@@H]2C[C@H]3CC[C@@H](C2)N3CCCc...
3,3,Cc1ccc(CN2[C@@H]3CC[C@H]2C[C@@H](C3)Oc4cccc(c4...,9.59,0.949,1,Cc1ccc(CN2[C@@H]3CC[C@H]2C[C@H](Oc2cccc(C(N)=O...
4,4,NC(=O)c1cccc(O[C@@H]2C[C@H]3CC[C@@H](C2)N3CCc4...,9.42,0.983,1,NC(=O)c1cccc(O[C@@H]2C[C@H]3CC[C@@H](C2)N3CCc2...


In [5]:
random_seed =100
start_time = str(time.ctime()).replace(':','-').replace(' ','_')
start = time.time()
radius = 3
T = 2
per_task_output_units_num = 2 # for classification model with 2 classes
output_units_num = len(tasks) * per_task_output_units_num # 2 

In [6]:
len(smiles_tasks_df)

7933

In [7]:
smilesList = [smiles for smiles in canonical_smiles_list if len(Chem.MolFromSmiles(smiles).GetAtoms())<101]

uncovered = [smiles for smiles in canonical_smiles_list if len(Chem.MolFromSmiles(smiles).GetAtoms())>100]

# cano_smiles안에 uncovered값과 겹치는 경우에 삭제
smiles_tasks_df = smiles_tasks_df[~smiles_tasks_df["cano_smiles"].isin(uncovered)]

feature_dicts = get_smiles_dicts(smilesList)
# keys = smiles값들이 cano_smiles값안에 있으면 두고 없다면 삭제
remained_df = smiles_tasks_df[smiles_tasks_df["cano_smiles"].isin(feature_dicts['smiles_to_atom_mask'].keys())] 
uncovered_df = smiles_tasks_df.drop(remained_df.index) 
uncovered_df # 최종적으로 uncovered_df에는 get_smiles_dicts로 바꾼 값과 cano_smiles와 비교하였을 때 삭제

Cc1ncoc1-c1nnc(SCCCN2CC3CC3(c3ccc(S(F)(F)(F)(F)F)cc3)C2)n1C
Cc1ncoc1-c1nnc(SCCCN2CC3CC3(c3cccc(S(F)(F)(F)(F)F)c3)C2)n1C
[Cl-]


,Unnamed: 0,SMILES,pIC50,hERG,Class,cano_smiles
1240,1240,Cc1ncoc1c2nnc(SCCCN3CC4CC4(C3)c5ccc(cc5)S(F)(F...,6.1,0.954,1,Cc1ncoc1-c1nnc(SCCCN2CC3CC3(c3ccc(S(F)(F)(F)(F...
1671,1671,Cc1ncoc1c2nnc(SCCCN3CC4CC4(C3)c5cccc(c5)S(F)(F...,5.9,0.950,1,Cc1ncoc1-c1nnc(SCCCN2CC3CC3(c3cccc(S(F)(F)(F)(...
3704,3704,[Cl-],5.2,0.972,1,[Cl-]


## Physicochemical properties 생성

In [8]:
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem import rdMolDescriptors

remained_df = remained_df.reset_index()

# mol 형태로 변환
train_mols = [Chem.MolFromSmiles(smiles) for smiles in remained_df["cano_smiles"]]

# mol 형태로 변환이 되지 않은 경우, none_list에 담는다
none_list = []
for i in range(len(train_mols)):
    if train_mols[i] is None :
        none_list.append(i)
        print('none_list에 추가됨')
    
reg_idx = 0
for i in none_list :
    del train_mols[i - reg_idx]
    reg_idx += 1
    
# none_list가 존재할 경우, 삭제 후 데이터프레임 인덱스 맞춰주기
if len(none_list) != 0 :
    remained_df = remained_df.drop(none_list, axis=0)
    remained_df = remained_df.reset_index(drop = True)

In [9]:
# fingerprint 생성
bit_info_list = [] # bit vector의 설명자 리스트 담기
bit_info = {} #bit vector 설명자
fps = []

b = 0
# mol 파일에서 fingerprint Bit Vector 형태로 변환하기
for a in train_mols :
    fps.append(AllChem.GetMorganFingerprintAsBitVect(a, 3, nBits = 1024, bitInfo = bit_info))
    bit_info_list.append(bit_info.copy()) # bit_info 그대로 가져오면 변수가 변해서 리스트 값이 달라지므로 .copy()
    
# array 변환

arr_list = []
for i in range(len(fps)):
    array = np.zeros((0,), dtype = np.int8)
    arr_list.append(array)
for i in range(len(fps)):
    bit = fps[i]
    DataStructs.ConvertToNumpyArray(bit, arr_list[i])
    
train_x = np.stack([i.tolist() for i in arr_list])
train_finprt = pd.DataFrame(train_x)
train_finprt.head()

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [10]:
import joblib
from sklearn.preprocessing import StandardScaler

# StandardScaler

sds_scaler = StandardScaler()

# molecular physicochemical properties 구하기

from rdkit.Chem import QED

train_qe = [QED.properties(mol) for mol in train_mols]
train_qe = pd.DataFrame(train_qe)
train_qe = train_qe.drop(columns = ['ROTB', 'AROM', 'ALERTS'])
train_qe[['MW', 'ALOGP', 'HBA', 'HBD', 'PSA']] = sds_scaler.fit_transform(train_qe[['MW', 'ALOGP', 'HBA', 'HBD', 'PSA']])
train_qe

,MW,ALOGP,HBA,HBD,PSA
0,-0.917145,-0.177788,-0.905935,-0.240880,-0.502855
1,1.267313,2.290658,-0.412761,-0.240880,-0.812253
2,-0.767714,-0.089251,-0.905935,-0.240880,-0.502855
3,-0.852917,-0.133940,-0.905935,-0.240880,-0.502855
4,-0.917145,-0.367382,-0.905935,-0.240880,-0.502855
...,...,...,...,...,...
7925,1.160303,-0.326243,0.573587,2.519498,1.530423
7926,1.181779,-0.166537,0.573587,2.519498,1.530423
7927,1.309733,-0.048112,0.573587,2.519498,1.530423
7928,1.331210,0.111594,0.573587,2.519498,1.530423


In [11]:
# 생성한 데이터 병합
input_df=pd.concat([train_finprt, train_qe], axis=1)
input_df.head()

,0,1,2,3,4,5,6,7,8,9,...,1019,1020,1021,1022,1023,MW,ALOGP,HBA,HBD,PSA
0,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,-0.917145,-0.177788,-0.905935,-0.24088,-0.502855
1,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1.267313,2.290658,-0.412761,-0.24088,-0.812253
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,-0.767714,-0.089251,-0.905935,-0.24088,-0.502855
3,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,-0.852917,-0.133940,-0.905935,-0.24088,-0.502855
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,-0.917145,-0.367382,-0.905935,-0.24088,-0.502855


In [12]:
new_remained_df = pd.concat([remained_df, input_df], axis = 1)
new_remained_df = new_remained_df.drop(columns = ['index', 'pIC50', 'hERG', 'Unnamed: 0'])
new_remained_df

,SMILES,Class,cano_smiles,0,1,2,3,4,5,6,...,1019,1020,1021,1022,1023,MW,ALOGP,HBA,HBD,PSA
0,Cc1ccc(CN2[C@@H]3CC[C@H]2C[C@@H](C3)Oc4cccc(c4...,1,Cc1ccc(CN2[C@@H]3CC[C@H]2C[C@H](Oc2cccc(C(N)=O...,1,0,0,0,0,0,0,...,1,0,0,0,0,-0.917145,-0.177788,-0.905935,-0.240880,-0.502855
1,COc1nc2ccc(Br)cc2cc1[C@@H](c3ccccc3)[C@@](O)(C...,1,COc1nc2ccc(Br)cc2cc1[C@@H](c1ccccc1)[C@@](O)(C...,0,1,1,1,0,0,0,...,0,0,0,0,0,1.267313,2.290658,-0.412761,-0.240880,-0.812253
2,NC(=O)c1cccc(O[C@@H]2C[C@H]3CC[C@@H](C2)N3CCCc...,1,NC(=O)c1cccc(O[C@@H]2C[C@H]3CC[C@@H](C2)N3CCCc...,0,0,0,0,0,0,0,...,1,0,0,0,0,-0.767714,-0.089251,-0.905935,-0.240880,-0.502855
3,Cc1ccc(CN2[C@@H]3CC[C@H]2C[C@@H](C3)Oc4cccc(c4...,1,Cc1ccc(CN2[C@@H]3CC[C@H]2C[C@H](Oc2cccc(C(N)=O...,0,0,0,0,0,0,0,...,1,1,0,0,0,-0.852917,-0.133940,-0.905935,-0.240880,-0.502855
4,NC(=O)c1cccc(O[C@@H]2C[C@H]3CC[C@@H](C2)N3CCc4...,1,NC(=O)c1cccc(O[C@@H]2C[C@H]3CC[C@@H](C2)N3CCc2...,0,0,0,0,0,0,0,...,1,0,0,0,0,-0.917145,-0.367382,-0.905935,-0.240880,-0.502855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7925,O=C1[C@H]2N(c3ccc(OCC=CCCNCC(=O)Nc4c(Cl)cc(cc4...,0,NC1=NCc2cc(Cl)c(c(Cl)c2)NC(=O)CNCCC=CCOc2ccc(c...,0,0,0,0,1,0,0,...,1,0,0,0,0,1.160303,-0.326243,0.573587,2.519498,1.530423
7926,O=C1[C@H]2N(c3ccc(OCCCCCNCC(=O)Nc4c(Cl)cc(cc4C...,0,NC1=NCc2cc(Cl)c(c(Cl)c2)NC(=O)CNCCCCCOc2ccc(cc...,0,0,1,0,1,0,0,...,1,0,0,0,0,1.181779,-0.166537,0.573587,2.519498,1.530423
7927,O=C1[C@H]2N(c3ccc(OCC=CCCCNCC(=O)Nc4c(Cl)cc(cc...,0,NC1=NCc2cc(Cl)c(c(Cl)c2)NC(=O)CNCCCC=CCOc2ccc(...,0,0,0,0,1,0,0,...,1,0,0,0,0,1.309733,-0.048112,0.573587,2.519498,1.530423
7928,O=C1[C@H]2N(c3ccc(OCCCCCCNCC(=O)Nc4c(Cl)cc(cc4...,0,NC1=NCc2cc(Cl)c(c(Cl)c2)NC(=O)CNCCCCCCOc2ccc(c...,0,0,1,0,1,0,0,...,1,0,0,0,0,1.331210,0.111594,0.573587,2.519498,1.530423


In [13]:
for key, value in feature_dicts.items():
    print(key)

smiles_to_atom_mask
smiles_to_atom_info
smiles_to_bond_info
smiles_to_atom_neighbors
smiles_to_bond_neighbors
smiles_to_rdkit_list


In [14]:
weights = []
for i,task in enumerate(tasks):    
    negative_df = new_remained_df[new_remained_df[task] == 0][["SMILES",task]] # 
    positive_df = new_remained_df[new_remained_df[task] == 1][["SMILES",task]] # 
    weights.append([(positive_df.shape[0]+negative_df.shape[0])/negative_df.shape[0],\
                    (positive_df.shape[0]+negative_df.shape[0])/positive_df.shape[0]])
# weights = [[2.862815884476534, 1.5368217054263567]]


# train/test/valid split
test_df = new_remained_df.sample(frac=1/10, random_state=random_seed) # test set
training_data = new_remained_df.drop(test_df.index) # training data
valid_df = training_data.sample(frac=1/9, random_state=random_seed) # validation set
train_df = training_data.drop(valid_df.index) # train set
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [15]:
print(train_df.shape)
print(valid_df.shape)
print(test_df.shape)

(6344, 1032)
(793, 1032)
(793, 1032)


In [16]:
print('train_df label의 shape')
print(train_df['Class'].value_counts())
print('test_df label의 shape')
print(test_df['Class'].value_counts())
print('valid_df label의 shape')
print(valid_df['Class'].value_counts())

train_df label의 shape
1    4102
0    2242
Name: Class, dtype: int64
test_df label의 shape
1    537
0    256
Name: Class, dtype: int64
valid_df label의 shape
1    521
0    272
Name: Class, dtype: int64


여기에서 test_df를 사용할 예정

In [17]:
best_param = {"roc_epoch": 0, "valid_roc": 0, "loss_epoch": 0, "valid_loss": float('inf')}
best_param


{'roc_epoch': 0, 'valid_roc': 0, 'loss_epoch': 0, 'valid_loss': inf}

In [18]:
# bayesian optimizer로 찾은 최적의 parameter 값 #
physicochemical_feature_dim = 1029
batch_size = 114
p_dropout = 0.475
fingerprint_dim = 100
learning_rate = 4.629
weight_decay = 2.291
radius = 3
T = 2

## Feature visualization

In [21]:
from AttentiveLayers_Viz import *

In [22]:
import os
os.getcwd()

'/home/dohyeon/hERGAT/dataset'

In [23]:
best_model = torch.load('../saved_model/model_hERG dataset'+'_'+str(82)+'.pt')     # best_param['roc_epoch']

best_model_dict = best_model.state_dict()
best_model_wts = copy.deepcopy(best_model_dict)

In [24]:
x_atom, x_bonds, x_atom_index, x_bond_index, x_mask, smiles_to_rdkit_list = get_smiles_array([smilesList[0]], feature_dicts)

num_atom_features = x_atom.shape[-1]
num_bond_features = x_bonds.shape[-1]

In [25]:
# 기존에 best_model_wts에 저장되어있는 가중치값들을 여기에 적용시킨다
model_for_viz = ExtendedFingerprint_viz(radius, T, num_atom_features, num_bond_features,fingerprint_dim, output_units_num, p_dropout, physicochemical_feature_dim)
model_for_viz.load_state_dict(best_model_wts)

<All keys matched successfully>

In [26]:
# 두 모델의 특정 layer에서 가중치가 동일한지 확인하는 코드
# 같다면 1을 출력
(best_model.align[0].weight == model_for_viz.align[0].weight).all()

tensor(True)

In [27]:
def eval_for_viz(model, viz_list):
    model.eval()
    test_acc_list = []
    test_roc_list = []
    mol_prediction_list = []
    atom_feature_list = []
    atom_weight_list = []
    mol_feature_list = []
    mol_feature_unbounded_list = []
    batch_list = []
    for i in range(0, len(viz_list), batch_size):
        batch = viz_list[i:i+batch_size]
        batch_list.append(batch) 
    for counter, batch in enumerate(batch_list):
        
        #batch_df = viz_list.loc[train_batch,:]
        #smiles_list = batch_df.cano_smiles.values
        x_atom, x_bonds, x_atom_index, x_bond_index, x_mask, smiles_to_rdkit_list = get_smiles_array(viz_list,get_smiles_dicts(batch))
        
        selected_rows = batch_df[batch_df['cano_smiles'] == smiles_list].drop(columns=['SMILES', 'Class', 'cano_smiles'])
        physicochemical_features= torch.tensor(selected_rows.to_numpy(dtype=np.float32), dtype=torch.float)
        
        atoms_prediction, atom_feature_viz, atom_attention_weight_viz, mol_feature_viz, mol_feature_unbounded_viz, mol_attention_weight_viz, mol_prediction = model(
            torch.Tensor(x_atom).cpu(), torch.Tensor(x_bonds).cpu(),
            torch.LongTensor(x_atom_index).cpu(),
            torch.LongTensor(x_bond_index).cpu(),
            torch.Tensor(x_mask).cpu())

        mol_prediction_list.append(mol_prediction.cpu().detach().squeeze().numpy())
        atom_feature_list.append(np.stack([atom_feature_viz[L].cpu().detach().numpy() for L in range(radius+1)]))
        atom_weight_list.append(np.stack([mol_attention_weight_viz[t].cpu().detach().numpy() for t in range(T)]))
        mol_feature_list.append(np.stack([mol_feature_viz[t].cpu().detach().numpy() for t in range(T)]))
        mol_feature_unbounded_list.append(np.stack([mol_feature_unbounded_viz[t].cpu().detach().numpy() for t in range(T)]))
        
    mol_prediction_array = np.concatenate(mol_prediction_list,axis=0)
    atom_feature_array = np.concatenate(atom_feature_list,axis=1)
    atom_weight_array = np.concatenate(atom_weight_list,axis=1)
    mol_feature_array = np.concatenate(mol_feature_list,axis=1)
    mol_feature_unbounded_array = np.concatenate(mol_feature_unbounded_list,axis=1)
#     print(mol_prediction_array.shape, atom_feature_array.shape, atom_weight_array.shape, mol_feature_array.shape)
    return mol_prediction_array, atom_feature_array, atom_weight_array, mol_feature_array, mol_feature_unbounded_array

In [28]:
viz_list = ['NCCc1ccc(S(=O)(=O)F)cc1']
    
# NCCc1ccc(S(=O)(=O)F)cc1    
#     'CCCCCCc1cccc(CCCCCC)[n+]1C',
# 'CCCCCCCN(CC)CCCCc1ccc(Cl)cc1',
# 'CCCCCC/C(C=O)=C\c1ccccc1',
# 'CCCCCCCN1CCC(Cc2ccccc2)CC1',
# 'CCCCCCCN(CC)CCCCCc1ccc(Cl)cc1',
# 'CCCCCCCN(CC)C/C=C/Cc1ccccc1'
    
mol_prediction_array, atom_feature_array, atom_weight_array, mol_feature_array, mol_feature_unbounded_array =  eval_for_viz(model_for_viz, viz_list)
x_atom, x_bonds, x_atom_index, x_bond_index, x_mask, smiles_to_rdkit_list = get_smiles_array(viz_list,get_smiles_dicts(viz_list))

NameError: name 'batch_df' is not defined

In [ ]:
feature_sorted = []
weight_sorted = []
figures = []

for i, smiles in enumerate(viz_list):
#     draw molecules in svg format
    atom_mask = x_mask[i]
    index_atom = smiles_to_rdkit_list[smiles]
    atom_feature = atom_feature_array[:, i]
    atom_weight = atom_weight_array[:, i]
    mol_prediction = mol_prediction_array[i]
    mol_feature = mol_feature_array[:, i]
    feature_list = []
    weight_list = []
    feature_reorder = []
    weight_reorder = []
    for j, one_or_zero in enumerate(atom_mask):
        if one_or_zero == 1.0:
            feature_list.append(atom_feature[:,j])
            weight_list.append(atom_weight[:,j])
            
    feature_reorder = np.stack([feature_list[m] for m in np.argsort(index_atom)])
    weight_reorder = np.stack([weight_list[m] for m in np.argsort(index_atom)])
#     reorder for draw
    if i == 0:    
        draw_index = [0, 1, 2,3,4,5,6,9, 12 ,11, 10, 7, 8]
        feature_reorder = np.stack([feature_reorder[m] for m in np.argsort(draw_index)])
        weight_reorder = np.stack([weight_list[m] for m in np.argsort(draw_index)])
#     elif i == 1:
#         draw_index = [1,2,3,4,5,6,7,8,9,10,11,12,13,0,14,15,16,17,18,19,20, 21, 22]
#         feature_reorder = np.stack([feature_reorder[m] for m in np.argsort(draw_index)])
#         weight_reorder = np.stack([weight_list[m] for m in np.argsort(draw_index)])
    else: # using rdkit index directly
        draw_index = list(range(len(index_atom)))
#     print(feature_reorder[0].shape,weight_list[0].shape)
    feature_sorted.append(feature_reorder)
    weight_sorted.append(weight_reorder)
    
    mol = Chem.MolFromSmiles(smiles)

    drawer = rdMolDraw2D.MolDraw2DSVG(400,400)
    drawer.SetFontSize(9)
    op = drawer.drawOptions()
    for index, re_index in enumerate(draw_index):
        op.atomLabels[index]=mol.GetAtomWithIdx(index).GetSymbol() + str(re_index)

    mol = rdMolDraw2D.PrepareMolForDrawing(mol)
    drawer.DrawMolecule(mol)
    drawer.FinishDrawing()
    svg = drawer.GetDrawingText()
    svg2 = svg.replace('svg:','')
    svg3 = SVG(svg2)
    display(svg3)
    
    img_path = f'figure/mol_{i}.svg'
    with open(img_path, 'w') as f:
        f.write(svg)
    
    
    intra_mol_correlation = [np.corrcoef(feature_reorder[:,L]) for L in range(radius+1)]
    sns.set(font_scale=2)
    
    for L in range(radius+1):
        plt.figure(dpi=150)
        fig, ax = plt.subplots(figsize=(13,11))
        mask = np.zeros_like(intra_mol_correlation[L])
        mask[np.triu_indices_from(mask)] = False
        sns.heatmap(np.around(intra_mol_correlation[L],1),cmap="YlGnBu", annot=True, ax=ax, mask=mask, square=True, annot_kws={"size": 16})
        figures.append(fig)
    plt.show()
    plt.close()
    
figure_count = 0

for fig in figures:
    fig.savefig(f'figure/figure_{figure_count}.png')
    figure_count += 1 